TODOs
- print tokens, cost
- use langsmith to evaluate prompts
- use langsmith to check agent flow -> time took etc

Challenges:
- large database with many tables
- query validation before run (guardrails)
- dynamic few shot examples
- finetuning datasets:  Spider, WikiSQL, CHASE, BIRD-SQL, or CoSQL.
- caching & monitoring

### Imports

In [37]:
# !pip install langchain langchain_experimental sqlglot --quiet

In [39]:
import boto3, json, sqlglot
from operator import itemgetter
from langchain.llms.bedrock import Bedrock
from langchain.sql_database import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.agents import Tool, AgentExecutor
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

from utils.prompts import (
    zero_shot_sql_prompt, 
    text_generate_prompt, text_generate_prompt_with_history,
    few_shot_sql_prompt, few_shot_sql_prompt_with_history,
    few_shot_sql_improved_prompt, few_shot_sql_builtin_prompt,
    agent_prompt_template, agent_prompt_template_with_history
)

### Utils

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={"max_tokens_to_sample": 512, "temperature": 0.0}
)

In [43]:
with open("/home/ubuntu/config.json") as file:
    config = json.load(file)

database_uri = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**config["rds_connect"])

rds_db = SQLDatabase.from_uri(database_uri=database_uri)

print(f"dialect: {rds_db.dialect}")
print(f"usable_table_names: {rds_db.get_usable_table_names()}")
print(f"table_info: {rds_db.get_table_info()}")

dialect: postgresql
usable_table_names: ['hoteldata']
table_info: 
CREATE TABLE hoteldata (
	room_type TEXT, 
	onsite_rate DOUBLE PRECISION, 
	room_amenities TEXT, 
	max_occupancy BIGINT, 
	roomdescription TEXT, 
	hotel_name TEXT, 
	city TEXT, 
	country TEXT, 
	star_rating BIGINT, 
	meals_included BOOLEAN
)

/*
3 rows from hoteldata table:
room_type	onsite_rate	room_amenities	max_occupancy	roomdescription	hotel_name	city	country	star_rating	meals_included
Vacation Home	636.09	Air conditioning: ;Closet: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Heating: ;High cha	4	Shower, Kitchenette, 2 bedrooms, 1 double bed or 2 single beds	Pantlleni	Beddgelert	United Kingdom	3	False
Vacation Home	591.74	Air conditioning: ;Closet: ;Dishwasher: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Heati	4	Shower, Kitchenette, 2 bedrooms, 1 double bed or 2 single beds	Willow Cottage	Beverley	United Kingdom	3	False
Guest room, Queen or Twin/Single Bed(s)	0.0	None	2	None	AC Hotel Manchester

In [44]:
def format_sql_query(raw_sql_query):
    sql_query = sqlglot.transpile(raw_sql_query, write='postgres', identify=True)[0]
    return sql_query

def get_db_response(x):
    sql_query = x["sql_query"]
    return rds_db.run(sql_query)    
    # return rds_db.run(sql_query, include_columns=True)

def text_to_sql_tool(question, chain):
    sql_query = chain.invoke({ "question": question })
    return sql_query

### 1. Text To SQL

#### zero-shot

In [45]:
zero_shot_sql_chain = zero_shot_sql_prompt | llm

In [46]:
%%time
query = "I want a hotel in Southern Europe and my budget is 200 bucks."
response = zero_shot_sql_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE country LIKE '%Southern Europe%' AND onsite_rate <= 200 LIMIT 5
CPU times: user 18 ms, sys: 3.02 ms, total: 21 ms
Wall time: 1.26 s


In [47]:
%%time
query = "Find a 2-person room in Vienna or London, preferably with meals included and AC"
response = zero_shot_sql_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE max_occupancy = 2 AND (city = 'Vienna' OR city = 'London') AND meals_included = true AND room_amenities LIKE '%AC%' LIMIT 5
CPU times: user 6.72 ms, sys: 449 µs, total: 7.16 ms
Wall time: 3.33 s


In [48]:
%%time
query = "A room with breakfast included for 3 people, at a Hilton"
response = zero_shot_sql_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE meals_included = true AND max_occupancy >= 3 AND hotel_name LIKE '%Hilton%' LIMIT 5
CPU times: user 7.46 ms, sys: 0 ns, total: 7.46 ms
Wall time: 7.22 s


In [49]:
%%time
query = "I want a hotel in the Balkans with a king sized bed and a hot tub. Budget is $300 a night."
response = zero_shot_sql_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE room_type LIKE '%king%' AND room_amenities LIKE '%hot tub%' AND onsite_rate <= 300 AND country LIKE '%Balkans%' LIMIT 5
CPU times: user 6.83 ms, sys: 0 ns, total: 6.83 ms
Wall time: 2.12 s


#### few-shot

##### observe: adding examples improves country column considering regions

In [50]:
# few_shot_sql_prompt.pretty_print()

In [51]:
few_shot_sql_chain = few_shot_sql_prompt | llm

In [52]:
%%time
query = "I want a hotel in Southern Europe and my budget is 200 bucks."
response = few_shot_sql_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE onsite_rate <= 200 AND (country LIKE '%Italy%' OR country LIKE '%Spain%' OR country LIKE '%Portugal%' OR country LIKE '%Greece%') LIMIT 5
CPU times: user 10.1 ms, sys: 306 µs, total: 10.4 ms
Wall time: 1.78 s


##### thought: add example values in a column

In [53]:
# few_shot_sql_improved_prompt.pretty_print()

In [54]:
few_shot_sql_improved_chain = few_shot_sql_improved_prompt | llm

In [55]:
%%time
query = "I want a hotel in Eastern Europe and my budget is 200 bucks."
response = few_shot_sql_improved_chain.invoke({
    "question": query
})
print(response)

 SELECT * FROM hoteldata WHERE onsite_rate <= 200 AND (country LIKE '%Poland%' OR country LIKE '%Czech Republic%' OR country LIKE '%Slovakia%' OR country LIKE '%Hungary%' OR country LIKE '%Romania%' OR country LIKE '%Bulgaria%') LIMIT 5
CPU times: user 7.18 ms, sys: 3.27 ms, total: 10.5 ms
Wall time: 3.58 s


### 2. Custom generate + execute SQL

#### chain

In [56]:
create_query = few_shot_sql_prompt | llm | format_sql_query

execute_summarize_query = text_generate_prompt | llm

final_inputs = RunnablePassthrough.assign(sql_query=create_query).assign(db_response=get_db_response)

full_chain = final_inputs | RunnablePassthrough.assign(output=execute_summarize_query)

In [57]:
%%time
query = "Show me top 3 most expensive hotels in London"
result = full_chain.invoke({"question": query})
# print(json.dumps(result, indent=2, default=str))

print(result["output"])

 The SQL query returns the top 3 most expensive hotels in London, ordered by onsite_rate descending. The most expensive is De Vere Beaumont Estate at £949.38 per night, followed by Balmoral Gardens at £695.64 and Sheraton Heathrow Hotel at £686.35.
CPU times: user 56.4 ms, sys: 3.09 ms, total: 59.5 ms
Wall time: 13.7 s


#### chain with memory

In [58]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    ai_prefix="A",
    human_prefix="H",
) 

In [59]:
create_query =  few_shot_sql_prompt_with_history | llm | format_sql_query

execute_summarize_query = text_generate_prompt_with_history | llm

full_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    ) 
    | RunnablePassthrough.assign(sql_query=create_query).assign(db_response=get_db_response) 
    | RunnablePassthrough.assign(output=execute_summarize_query)
)

In [61]:
%%time
query = "Show me top 3 most expensive hotels in London"
result = full_chain.invoke({"question": query})
print(result["output"])

 The top 3 most expensive hotels in London are De Vere Beaumont Estate at $949.38 per night with a 4 star rating, Balmoral Gardens at $695.64 per night also with a 4 star rating, and Sheraton Heathrow Hotel at $686.35 per night with a 4 star rating.
CPU times: user 73.6 ms, sys: 5.13 ms, total: 78.8 ms
Wall time: 9.49 s


In [62]:
memory.chat_memory.add_user_message(result["question"])
memory.chat_memory.add_ai_message(result["output"])

memory.load_memory_variables({})

{'chat_history': 'H: Show me top 3 most expensive hotels in London\nA:  The top 3 most expensive hotels in London are De Vere Beaumont Estate at $949.38 per night with a 4 star rating, Balmoral Gardens at $695.64 per night also with a 4 star rating, and Sheraton Heathrow Hotel at $686.35 per night with a 4 star rating.'}

In [63]:
%%time
query = "What are the room amenties available of above hotels ? "
result = full_chain.invoke({"question": query})
print(result["output"])

 The room amenities available at the top 3 most expensive London hotels are air conditioning, free Wi-Fi, safe boxes, and other standard amenities. De Vere Beaumont Estate also offers wake-up service. Balmoral Gardens has full kitchens. Sheraton Heathrow has bathrobes and slippers.
CPU times: user 78.6 ms, sys: 0 ns, total: 78.6 ms
Wall time: 7.29 s


In [64]:
memory.chat_memory.add_user_message(result["question"])
memory.chat_memory.add_ai_message(result["output"])

memory.load_memory_variables({})

{'chat_history': 'H: Show me top 3 most expensive hotels in London\nA:  The top 3 most expensive hotels in London are De Vere Beaumont Estate at $949.38 per night with a 4 star rating, Balmoral Gardens at $695.64 per night also with a 4 star rating, and Sheraton Heathrow Hotel at $686.35 per night with a 4 star rating.\nH: What are the room amenties available of above hotels ? \nA:  The room amenities available at the top 3 most expensive London hotels are air conditioning, free Wi-Fi, safe boxes, and other standard amenities. De Vere Beaumont Estate also offers wake-up service. Balmoral Gardens has full kitchens. Sheraton Heathrow has bathrobes and slippers.'}

#### agent

In [65]:
create_query = few_shot_sql_prompt | llm | format_sql_query

# execute_summarize_query = text2sql_chain # when don't want to execute queries 
execute_summarize_query = (
    RunnablePassthrough.assign(sql_query=create_query).assign(db_response=get_db_response)
    | text_generate_prompt 
    | llm
)

In [66]:
tools = [
    Tool(
        name="run_text_to_sql",
        func=lambda question: text_to_sql_tool(question, execute_summarize_query),
        description=(
            "Use when you are asked analytical questions about hotels, stay and other facilities provided."
            " The input should be the question itself."
        ),
    )
]

In [67]:
agent_prompt = PromptTemplate(
    input_variables = ["question", "agent_scratchpad"],
    partial_variables = {
        "tools": render_text_description(tools),
        "tool_names": ", ".join([t.name for t in tools]),
    },
    template = agent_prompt_template,
)

# agent_prompt.pretty_print()

In [68]:
agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | agent_prompt
    | llm.bind(stop=["\nObservation"])
    | ReActJsonSingleInputOutputParser()
)

In [69]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [70]:
agent_executor.invoke({
    "question": "How many hotels in London ?"
})



> Entering new AgentExecutor chain...
 Thought: I will use run_text_to_sql to get the number of hotels in London.

Action:
```
{
  "action": "run_text_to_sql",
  "action_input": "How many hotels are there in London?"
}
```

Action Input: How many hotels are there in London?

The SQL query counts the number of hotels in the hoteldata table where the city contains 'London'. The response indicates there are 475 hotels in London.
Final Answer: There are 475 hotels in London based on the hotel data.

> Finished chain.


{'question': 'How many hotels in London ?',
 'output': 'There are 475 hotels in London based on the hotel data.'}

#### agent with memory

In [71]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    ai_prefix="A",
    human_prefix="H",
) 

In [72]:
create_query =  few_shot_sql_prompt_with_history | llm | format_sql_query

execute_summarize_query = (
    RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")) 
    | RunnablePassthrough.assign(sql_query=create_query).assign(db_response=get_db_response) 
    | text_generate_prompt_with_history 
    | llm
)

In [73]:
tools = [
    Tool(
        name="run_text_to_sql",
        func=lambda question: text_to_sql_tool(question, execute_summarize_query),
        description=(
            "Use when you are asked analytical questions about hotels, stay and other facilities provided."
            " The input should be the question itself."
        ),
    )
]

In [74]:
agent_prompt_with_history = PromptTemplate(
    input_variables = [
        "chat_history",
        "question", 
        "agent_scratchpad"
    ],
    partial_variables = {
        "tools": render_text_description(tools),
        "tool_names": ", ".join([t.name for t in tools]),
    },
    template = agent_prompt_template_with_history,
)

In [75]:
agent_with_history = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"]
    }
    | agent_prompt_with_history
    | llm.bind(stop=["\nObservation"])
    | ReActJsonSingleInputOutputParser()
)

In [76]:
agent_executor_with_history = AgentExecutor(
    agent=agent_with_history,
    tools=tools,
    verbose=True,
    memory=memory, # At the start, memory loads variables and passes them along in the chain. At the end, it saves any returned variables. 
    handle_parsing_errors=True,
)

In [77]:
memory.load_memory_variables({})

{'chat_history': ''}

In [78]:
%%time
# query = "Show me top 3 most expensive hotels in London"
query = "How many hotels in London ?"
response = agent_executor_with_history.invoke({ "question": query })
print(response["output"])



> Entering new AgentExecutor chain...
 Thought: I will use run_text_to_sql to get the number of hotels in London.

Action:
```
{
  "action": "run_text_to_sql",
  "action_input": "SELECT COUNT(*) FROM hotels WHERE city = 'London'"
}
```

Action Input: SELECT COUNT(*) FROM hotels WHERE city = 'London'
 The SQL query counted 475 rows in the hoteldata table where the city column contains 'London'.
Final Answer: There are 475 hotels in London.

> Finished chain.
There are 475 hotels in London.
CPU times: user 152 ms, sys: 6.43 ms, total: 158 ms
Wall time: 15.9 s


In [79]:
%%time
query = "What are the room amenties available of above hotels ?"
response = agent_executor_with_history.invoke({ "question": query })
print(response["output"])



> Entering new AgentExecutor chain...
 Thought: I need to analyze the hotel data to find information about room amenities of the hotels in London.

Action:
```
{
  "action": "run_text_to_sql",
  "action_input": "What are the room amenities available at the 475 hotels in London?"  
}
```

Action Input: What are the room amenities available at the 475 hotels in London?
 The query returns a list of all the distinct room amenities available across the 475 hotels in London. The amenities include things like air conditioning, coffee/tea makers, WiFi, TVs, hair dryers, and more.
Final Answer: The room amenities available across the 475 hotels in London include air conditioning, coffee/tea makers, WiFi, TVs, hair dryers, and various other standard hotel room amenities. The full list contains numerous distinct amenities across all the London hotels.

> Finished chain.
The room amenities available across the 475 hotels in London include air conditioning, coffee/tea makers, WiFi, TVs, hair drye

### 3. Builtin generate + execute SQL

#### chain

In [80]:
from langchain_experimental.sql import SQLDatabaseChain

sql_db_chain = SQLDatabaseChain.from_llm(
    llm=llm,
    db=rds_db,
    prompt=few_shot_sql_builtin_prompt,
    # use_query_checker=True,
    return_intermediate_steps=True,
    verbose=True,
)

question = "show me top 3 most expensive hotels in Italy"

response = sql_db_chain(question)
response



> Entering new SQLDatabaseChain chain...
show me top 3 most expensive hotels in Italy
SQLQuery:SELECT * FROM hoteldata WHERE country LIKE '%Italy%' ORDER BY onsite_rate DESC LIMIT 3
SQLResult: [('Presidential Suite', 2190.1, 'Air conditioning: ;Alarm clock: ;Bathrobes: ;Closet: ;Coffee/tea maker: ;Desk: ;Free Wi-Fi in all rooms!: ;Hair dryer: ;Heating: ;In-room safe box: ;Linens: ;Mini bar: ;Mirror: ;Satellite/cable channels: ;Seating area: ;Slippers: ;Smoke detector: ;Sofa: ;Soundproofing: ;Telephone: ;Toiletries:...', 1, 'Room size: 60 m²/646 ft², Non-smoking, 2 bathrooms, Shower, 2 king beds', 'Splendid Venice Venezia - Starhotels Collezione', 'Venice', 'Italy', 4, True), ('One-Bedroom Apartment', 1105.09, 'Air conditioning: ;Closet: ;Desk: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Hair dryer: ;Heating: ;In-room safe box: ;Kitchenware: ;Private entrance: ;Refrigerator: ;Satellite/cable channels: ;Seating area: ;Separate dining area: ;Shower: ;Sofa: ;Telephone: ;Toiletries: ;Towel

{'query': 'show me top 3 most expensive hotels in Italy',
 'result': "SELECT * FROM hoteldata WHERE country LIKE '%Italy%' ORDER BY onsite_rate DESC LIMIT 3",
 'intermediate_steps': [{'input': 'show me top 3 most expensive hotels in Italy\nSQLQuery:',
   'top_k': '5',
   'dialect': 'postgresql',
   'table_info': '\nCREATE TABLE hoteldata (\n\troom_type TEXT, \n\tonsite_rate DOUBLE PRECISION, \n\troom_amenities TEXT, \n\tmax_occupancy BIGINT, \n\troomdescription TEXT, \n\thotel_name TEXT, \n\tcity TEXT, \n\tcountry TEXT, \n\tstar_rating BIGINT, \n\tmeals_included BOOLEAN\n)\n\n/*\n3 rows from hoteldata table:\nroom_type\tonsite_rate\troom_amenities\tmax_occupancy\troomdescription\thotel_name\tcity\tcountry\tstar_rating\tmeals_included\nVacation Home\t636.09\tAir conditioning: ;Closet: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Heating: ;High cha\t4\tShower, Kitchenette, 2 bedrooms, 1 double bed or 2 single beds\tPantlleni\tBeddgelert\tUnited Kingdom\t3\tFalse\nVacation Home\

#### agent

In [81]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(
    llm=llm, 
    db=rds_db,
    agent_type="zero-shot-react-description", 
    verbose=True
)
question = "show me top 3 most expensive hotels in Italy"

response = agent_executor.invoke(question)
response




> Entering new SQL Agent Executor chain...
 Here is how I would answer the question to show the top 3 most expensive hotels in Italy:

Thought: I first need to understand what tables are available in the database.

Action: sql_db_list_tables
Action Input: 
hoteldata Here is how I would go about answering the question to show the top 3 most expensive hotels in Italy:

First, list the available tables:

Action: sql_db_list_tables  
Action Input:  
hoteldata Here is the full solution to show the top 3 most expensive hotels in Italy:

Thought: I first need to understand the schema of the hoteldata table to know which fields to query.

Action: sql_db_schema  
Action Input: hoteldata

CREATE TABLE hoteldata (
	room_type TEXT, 
	onsite_rate DOUBLE PRECISION, 
	room_amenities TEXT, 
	max_occupancy BIGINT, 
	roomdescription TEXT, 
	hotel_name TEXT, 
	city TEXT, 
	country TEXT, 
	star_rating BIGINT, 
	meals_included BOOLEAN
)

/*
3 rows from hoteldata table:
room_type	onsite_rate	room_amenitie

{'input': 'show me top 3 most expensive hotels in Italy',
 'output': 'The top 3 most expensive hotels in Italy are:\n\n1. Splendid Venice Venezia - Starhotels Collezione, Venice, $2190.10 per night \n2. Corte Family Residence, Borca di Cadore, $1105.09 per night\n3. Hotel Royal, Catania, $1064.08 per night'}

In [84]:
print(response["output"])

The top 3 most expensive hotels in Italy are:

1. Splendid Venice Venezia - Starhotels Collezione, Venice, $2190.10 per night 
2. Corte Family Residence, Borca di Cadore, $1105.09 per night
3. Hotel Royal, Catania, $1064.08 per night


### Test

In [82]:
# query = "Show me top 3 expensive hotels in london"

# async for chunk in full_chain.astream_events({"question": query}, version="v1",):
#     # print(chunk, end="", flush=True)
#     print(chunk)

In [83]:
# query = "Show me top 3 expensive hotels in london"

# async for chunk in full_chain.astream({"question": query}):
#     if "sql_query" in chunk:
#         print(chunk["sql_query"])
#     elif "output" in chunk:
#         print(chunk["output"])